## 영상 분할

1. 영상 분할은 입력 영상에서 관심 있는 영역을 분리하는 과정이다.
2. 이러한 영상 분할은 영상분석, 물체 인식, 추적 등 대부분의 영상처리 응용에서 필수적인 단계이다.
3. 영상 분할은 크게 경계선 또는 영역으로 분할한다.
4. 임계값 사용 방법은 가장 간단한 영상 분할 방법으로 5장에서 cv2.threshold(), cv2.adaptiveThreshold()를 이미 설명하였다.

### Canny 에지 검출

1. 에지는 영상의 물체와 물체 사이 또는 물체와 배경사이의 테두리에서 발생한다.
2. 6장의 1차 미분 필터 cv2.Sobel()와 2차 미분 필터 cv2.Laplacian() 또는 LoG로 필터링한 후에 0-교차점을 찾아 에지를 검출할 수 있다.

- cv2.Canny(image, threshold1, threshold2[, edges[, apertureSize[, L2gradient ]]])

##### 1-채널 8비트 입력 영상 image에서 에지를 검출하여 edges를 반환한다.
##### apertureSize는 그래디언트를 계산하기 위한 Sobel 필터의 크기로 사용하고, 히스테리시스 임계값에 사용되는 두 임계값 threshold1, threshold2를 사용하여 연결된 에지를 얻는다.
##### 먼저 높은 값의 임계값을 사용하여 그래디언트 방향에서 낮은 값의 임계값이 나올 때까지 추적하며 에지를 연결하는 히스테리시스 임계값 방식을 사용한다.
##### L2gradient = True이면 그래디언트의 크기를 √((dI/dx)²+(dI/dy)²)로 계산하고, L2gradient = False이면 그래디언트의 크기를 |dI/dx|+|dI/dy|로 계산한다.
##### 입력 영상에 cv2.Canny()함수 호출전에 cv2.GaussianBlur()함수 같은 블러링 함수로 영상을 부드럽게 하면 잡음에 덜 민감할 수 있다.

#### 실습

In [4]:
# 에지 검출: cv2.Canny()
# 0701.py
import cv2
import numpy as np

src = cv2.imread('./data/lena.jpg', cv2.IMREAD_GRAYSCALE)

edges1 = cv2.Canny(src, 50, 100) # 입력 영상 src에서 threshold1 = 50, threshold2 = 100, apertureSize = 3으로 Canny 에지를 edges1에 검출한다.
edges2 = cv2.Canny(src, 50, 200) # 입력 영상 src에서 threshold1 = 50, threshold2 = 200, apertureSize = 3으로 Canny 에지를 edges2에 검출한다.
 
cv2.imshow('edges1',  edges1)
cv2.imshow('edges2',  edges2)
cv2.waitKey()
cv2.destroyAllWindows()

### Hough 변환에 의한 직선 및 원 검출

1. Sobel, LoG, Canny로 검출한 에지는 단순히 화소들의 집합니다. 즉, 에지는 직선, 사각형, 원, 곡선 등의 구조적 정보를 갖지 않는다.
2. Hough 변환을 사용하면 에지에서 직선 또는 원의 방정식의 파라미터를 검출할 수 있다.
3. 직선 검출을 위한 퍼르 변환 알고리즘은 각 에지 점 (x, y)에 대하여, 이산 격자 간격에서 점(x, y)를 지나가는 가능한 모든 직선의 방정식의 파라미터(ρx,θh)를 계산하여, 대응하는 어큐뮬레이터 정수 배열을 1씩 증가시킨다.
4. 모든 에지점을 이처럼 처리하면 A(k,h)에는 (ρx,θh)인 직선 위에 있는 에지의 개수가 누적된다. A(k, h) > threshold인 모든(k, h)중에서 지역 극값인 직선을 찾는다.
5. 배열 A(k, h)의 각 위치는 하나의 직선의 방정식 ρx = xcos(θh) + ysin(θh)을 표현한다.

- cv2.HoughLines(image, tho, theta, threshold[, lines[, srn[, stn[, min_theta ]]]]])

##### 1-채널 8비트 이진영상 image에서 직선을 lines에 검출한다.
##### lines는 검충된 직선의 (ρ,θ)가 저장된 배열이다. rho는 원점으로부터의 거리 간격, theta는 x축과의 각도로 라디안 간격, threshold는 직선을 검출하기 위한 어큐뮬레이터의 임계값이다.

- cv2.HoughLinesP(image, tho, theta, threshold[, lines[, minLineLength[, maxLineGap ]]])

##### probabilistic Hough 변환을 이용하여 양 끝점이 있는 선분을 lines에 검출한다.
##### 출력 lines는 선분의 양 끝점(x1, x1, x2, y2)를 저장하는 배열이다. rho는 원점으로부터의 거리 간격, theta는 x축과의 각도로 라디안 간격, threshold는 직선을 검출하기 위한 어큐뮬레이터의 임계값이다.
##### minLineLength는 검출한 최소 직선의 길이이고, maxLineGap은 직선 위의 에지들의 최대 허용 간격이다.

- cv2.HoughCircles(image, method, dp, minDise[, circles[, param1[, param2[, minnRadius[, maxRadius ]]]]])

##### 1채널 8비트 그레이스케일 영상(에지가 아니다) image에서 원을 찾아, 원의 매개변수 (cx, cy, r)를 저장한 배열 circles를 반환한다.
##### 현재는 method = cv2.HOUGH_GRADIENT 방법만이 구현되어 있다. dp는 어큐뮬레이터 간격에서 영상 간격으로의 역 비율이다.
##### dp = 1이면 어큐뮬레이터가 입력 영상과 같은 해상도를 갖고, dp = 2이면 어큐뮬레이터의 크기가 영상 가로크기의 반, 세로 크기의 반을 의미한다.
##### minDist는 검출된 원의 중심 사이의 최소 거리로 너무 작으면 실제 원 주위에 너무 많은 원이 검출되고, 너무 크면 검출하지 못하는 원이 있을 수 있다.
##### param1은 Canny에서 검출의 높은 임계값인 threshold2이다. 낮은 임계값은 threshold1 = param1 / 2이다.
##### param2는 원 검출을 위한 어큐뮬레이터의 임계값으로 작은 값이면 너무 많은 원이 검출되고, 너무 크면 찾지 못하는 원이 있을 수 있다.
##### minRadius는 원의 최소 반지름, maxRadius는 원의 최대 반지름이다.

#### 실습

In [9]:
## 직선검출:cv2.HoughLines()
# 0702.py
import cv2
import numpy as np

src = cv2.imread('./data/rect.jpg')
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 100)
lines = cv2.HoughLines(edges, rho=1, theta=np.pi/180.0, threshold=100) # cv2.HoughLines()로 rho = 1, theta = np.pi / 180.0, threshold = 100을 
                                                                       # 적용하여 선분을 lines에 검출한다. 검출된 직선의 rho, theta를 저장한
                                                                       # lines의 배열의 모양은 line.shape = (4, 1, 2)이다.
                                                                       # 4개의 직선의 rho, theta를 저장한 (1, 2)로 이해하면 된다.
print('lines.shape=', lines.shape)

for line in lines:
    rho, theta   = line[0]
    c = np.cos(theta)
    s = np.sin(theta)
    x0 = c*rho
    y0 = s*rho
    x1 = int(x0 + 1000*(-s))
    y1 = int(y0 + 1000*(c))
    x2 = int(x0 - 1000*(-s))
    y2 = int(y0 - 1000*(c))
    cv2.line(src, (x1, y1), (x2, y2), (0,0,255), 2)
    
# for 문에서 각 직선의 매개변수는 rho, theta = line[0]이고, rho, theta를 이용하여 검출된 직선을 그린다. 원점에서 (rho, theta)에 의한 직선과 직각으로
# 만나는 좌표(x0, y0)는 x0 = rho *c, y0 = rho * s로 계산한다. 직선 방향으로의 단위 벡터는 (cos(theta), -sin(theta))이다.
# 이 단위 벡터를 +, -방향으로 스케일링하고, x0, y0에 더하여 선분의 양 끝점 (x1, y1)과 (x2, y2)를 계산하여 cv2.line()으로 src에 직선을 그리면 그림과 같이 4개의 직선이 표시된다.
    
cv2.imshow('edges',  edges)
cv2.imshow('src',  src)
cv2.waitKey()
cv2.destroyAllWindows()


lines.shape= (4, 1, 2)


In [11]:
## 선분 검출: cv2.HoughLinesP()
# 0703.py
import cv2
import numpy as np

src = cv2.imread('./data/rect.jpg')
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 100)
lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi/180.0, threshold=100) # cv2.HoughLinesP()로 rho = 1, theta = np.pi / 180.0, threshold = 100을
                                                                        # 적용하여 선분을 lines에 검출한다. 검출된 선분의 양 끝점(x1, y1, x2, y2)을
                                                                        # 저장한 lines 배열의 모양은 lines.shape = (4, 1, 4)이다. 4개의 선분의양 끝점을 저장한 (1, 4)로 이해하면 된다.
print('lines.shape=', lines.shape)

for line in lines:
    x1, y1, x2, y2   = line[0]
    cv2.line(src,(x1,y1),(x2,y2),(0,0,255),2)
    
cv2.imshow('edges',  edges)
cv2.imshow('src',  src)
cv2.waitKey()
cv2.destroyAllWindows()


lines.shape= (4, 1, 4)


In [41]:
# 0704.py
import cv2
import numpy as np

#1
src1 = cv2.imread('./data/circles.jpg')
gray1 = cv2.cvtColor(src1,cv2.COLOR_BGR2GRAY)
circles1 = cv2.HoughCircles(gray1, method = cv2.HOUGH_GRADIENT,
            dp=1, minDist=50, param2 = 15)

circles1 =  np.int32(circles1)
print('circles1.shape=', circles1.shape)
for circle in circles1[0,:]:    
    cx, cy, r  = circle
    cv2.circle(src1, (cx, cy), r, (0,0,255), 2)
cv2.imshow('src1',  src1)

# 'circles.jpg'의 그레이스케일 영상 gray1에서, cv2.HoughCircles()로 method = cv2.HOUGH_GRADIENT, dp = 1, minDist = 50, param2 = 15를 적용하여
# circles1에 원을 검출한다. param2 = 15는 원 검출을 위한 어큐뮬레이터의 임계값으로 원 위의 에지 개수이다. 검출된 원의 중심 cx, cy, 반지름 r을 저장한
# circles1 배열은 circles1.shape = (1, 3, 3)으로 circles1[0]의 3개의 행에 원의 cx, cy, r을 저장한다.

#2
src2 = cv2.imread('./data/circles2.jpg')
gray2 = cv2.cvtColor(src2,cv2.COLOR_BGR2GRAY)
circles2 = cv2.HoughCircles(gray2, method = cv2.HOUGH_GRADIENT,
          dp=1, minDist=50, param2=15, minRadius=30, maxRadius=100)

# 'circle.jpg'의 그레이스케일 영상 gray2에서, cv2.HoughCircle()로 method = cv2.HOUGH_GRADIENT, dp = 1, minDist = 50, param2 = 15, minRadius = 30,
# maxRadius = 100을 적용하여 circle2에 원을 검출한다. 원의 반지름의 범위를 minRadius = 30, maxRadius = 100으로 제한하여 원을 검출한다.
# circles2 배열은 circles2.shape = (1, 6, 3)으로 circles[0]의 6개의 행에 원의 cx, cy, r을 저장한다.

circles2 =  np.int32(circles2)
print('circles2.shape=', circles2.shape)
for circle in circles2[0,:]:    
    cx, cy, r  = circle
    cv2.circle(src2, (cx, cy), r, (0,0,255), 2) 
cv2.imshow('src2',  src2)
cv2.waitKey()
cv2.destroyAllWindows()


circles1.shape= (1, 3, 3)
circles2.shape= (1, 6, 3)


### 컬러 범위에 의한 영역 분할

- cv2.inRange(src, lowerb, upperb[, dst ])

##### 입력 src의 각 화소가 lowerb(i) <= src(i) <= upperb(i) 범위에 있으면, dst(i) = 255, 그렇지 않으면 dst(i) = 0이다.
##### lowerb와 upperb는 Scalar도 가능하고, dst는 src와 같은 크기의 8비트 부호없는 정수 자료형이다. src가 다중 채널인 경우, 모든 채널에 대하여 범위를 만족해야 한다.
##### 3-채널 컬러 영상에서, HSV 색상으로 변환한 후에, 색상 범위를 지정하여 손, 얼굴 등의 피부 검출 등을 분할할 때 유용하다.

#### 실습

In [19]:
## 컬러 영역 검출: cv2.inRange()
# 0705.py
import cv2
import numpy as np

#1
src1 = cv2.imread('./data/hand.jpg')
hsv1 = cv2.cvtColor(src1, cv2.COLOR_BGR2HSV)
lowerb1 = (0, 40, 0)
upperb1 = (20, 180, 255)
dst1 = cv2.inRange(hsv1, lowerb1, upperb1)   # 'hand.jpg'영상을 3-채널 BGR 컬러 영상으로 읽은 src1을 HSV영상으로 hsv1에 변환하고, hsv1에서 cv2.inRange()로
                                             # lowerb1 = (0, 40, 0), upperb1 = (20, 180, 255) 범위를 적용하여 손영역을 분할한다.

#2
src2 = cv2.imread('./data/flower.jpg')
hsv2 = cv2.cvtColor(src2,cv2.COLOR_BGR2HSV)
lowerb2 = (150, 100, 100)
upperb2 = (180, 255, 255)
dst2 = cv2.inRange(hsv2, lowerb2, upperb2)   # 'flower.jpg'영상을 3-채널 BGR 컬러 영상으로 읽은 src2를 HSV영상으로 hsv2에 변환하고, hsv2에서 cv2.inRange()로
                                             # lowerb2, = (150, 100, 100), upperb2 = (180, 255, 255) 범위를 적용하여 꽃 영역을 분할한다.

#3
cv2.imshow('src1',  src1)
cv2.imshow('dst1',  dst1)
cv2.imshow('src2',  src2)
cv2.imshow('dst2',  dst2)
cv2.waitKey()
cv2.destroyAllWindows()


### 윤곽선 검출 및 그리기

1. 윤곽선을 검출하고 그리기 기능을 수행 함수로는 물체의 경계를 이루고 있는 윤곽선을 검출하는 findContours() 함수와 검출된 윤곽선을 영상에 그리는 drawContours()함수가 있다.
2. 입력 영상은 cv2.inRange(), cv2.threshold(), cv2.Canny() 등을 사용하여 얻은 이진 영상이다.

- cv2.findContours(image, mode, method[, contours[, hierarchy[, offset ]]]) -> contours, hierarchy

##### 1-채널 8비트 영상 image에서 윤곽선을 검출한다. image는 0이 아닌 값은 1로 취급하는 영상이다.
##### contours는 검출된 윤곽선이고, 리스트 자료형의 각 요소에 윤곽선을 배열로 반환한다.
##### hierarchy는 윤곽선의 계층 구조에 관한 출력 배열이다. hierarchy[0][i]는 윤곽선 contour[i]에 대한 계층 구조의 배열이다.
##### hierarchy[0][i][0]과 hierarchy[0][i][1]은 같은 계층 구조 에벨에서 다음과 이전 윤곽선이다. hierarchy[0][i][2]와 hierarchy[0][i][3]은 첫 번째 자식과 부모 윤곽선이다. 대응하는 윤곽선이 없으면 음수 값을 갖는다.
##### mode는 윤곽선의 검색모드로 사용 가능한 mode는 cv2.RETR_EXTERMAL, cv2.RETR_LIST, cv2.RETR_CCOMP, cv2.RETR_TREE이다.
##### method는 윤곽선의 근사 방법으로 사용 가능한 method는 cv2.CHAIN_APPROX_NONE, cv2.CHAIN_APPROX_SIMPLE, cv2.CHAIN_APPROX_TC89_L1, cv2.CHAIN_APPROX_TC89_KCOS이다.

- cv2.drawContours(image, contours, contourIdx, color[, thickness[, lineType[, hierarchy[, maxLevel[, offset ]]]]]) -> image

##### 영상 image에 윤곽선 리스트 contours를 color 색상으로 그린다. 각각의 윤곽선 contour는 좌표 배열이다.
##### contourIdx는 표시할 윤곽선 첨자로 contourIdx < 0 이면 모든 윤곽선을 그린다.
##### thickness는 윤곽선의 두께이며, thickness = cv2.FILLED(-1)이면 윤곽선 내부를 채운다.
##### lineType은 라인의 형태로 8, 4, CV_AA 중 하나이다.
##### hierarchy는 윤곽선의 계층 구조로 maxLevel에 의해 주어진 계층 구조를 그릴 때 사용된다.
##### maxLevel = 0이면 명시된 contour만을 그리고, maxLebel = 1이면 contour를 그린 뒤에 contour에 내포된 윤곽선을 그린다.
##### offset에 주어진 좌표만큼 윤곽선의 모든 좌표를 이동시킨다.

In [18]:
## 윤곽선 검출 및 그리기 1: mode = cv2.RETR_EXTERNAL
# 0706.py
import cv2
import numpy as np

#1
src = np.zeros(shape=(512,512,3), dtype=np.uint8)
cv2.rectangle(src, (50, 100), (450, 400), (255, 255, 255), -1)
cv2.rectangle(src, (100, 150), (400, 350), (0, 0, 0), -1)
cv2.rectangle(src, (200, 200), (300, 300), (255, 255, 255), -1)
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
# src에 512 x 512크기의 3-채널 컬러영상을 생성하고, cv2.rectangle()로 채워진 사각형을 그린다.
# cv2.cvtColor()로 그레이스케일 영상 gray으로 변환한다.

#2
mode = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE
## method =cv2.CHAIN_APPROX_NONE
contours, hierarchy = cv2.findContours(gray, mode, method)
# cv2.findContours()로 윤곽선 contours를 검출한다. 
# mode = cv2.RETR_EXTERNAL로 리스트 contours에 len(contours) = 1개의 가장 외각의 윤곽선을 검출한다.
# method = cv2.CHAIN_APPROX_SIMPLE로 윤곽선을 다각형으로 근사한 과표를 반환한다. contours[0].shape = (4, 1, 2)은 4개의 검출된 좌표가 (1, 2) 배열에 저장된다.
# method = cv2.CHAIN_APPROX_NONE이면, contours[0].shape = (1400, 1, 2)로 윤곽선위의 모든 좌표 1400개를 검출한다.

print('type(contours)=', type(contours))
print('type(contours[0])=', type(contours[0]))
print('len(contours)=', len(contours))
print('contours[0].shape=', contours[0].shape)
print('contours[0]=', contours[0])

#3
cv2.drawContours(src, contours, -1, (255,0,0), 3) # contourIdx < 0 이면 모든 윤곽선을 그리는데 -1이기때문에 모든 윤곽선을 그린다.
# cv2.drawContours(src, contours, -1, (255, 0, 0), 3)는 검출된 윤곽선 contours를 전부를 src(255, 0, 0) 컬러로 두께 3으로 그린다.

#4
for pt in contours[0][:]: 
    cv2.circle(src, (pt[0][0], pt[0][1]), 5, (0,0,255), -1)
# for문으로 윤곽선의 (1, 2)좌표 배열 pt에 의해 중심점(pt[0][0], pt[0][1]), 반지름 5인 (0, 0, 255) 컬러로 채워진 원을 src에 그린다.
    
cv2.imshow('src',  src)
cv2.waitKey()
cv2.destroyAllWindows()


type(contours)= <class 'tuple'>
type(contours[0])= <class 'numpy.ndarray'>
len(contours)= 1
contours[0].shape= (4, 1, 2)
contours[0]= [[[ 50 100]]

 [[ 50 400]]

 [[450 400]]

 [[450 100]]]


In [20]:
## 윤곽선 검출 및 그리기 2: mode = cv2.RETR_LIST
# 0707.py
import cv2
import numpy as np

#1
src = np.zeros(shape=(512,512,3), dtype=np.uint8)
cv2.rectangle(src, (50, 100), (450, 400), (255, 255, 255), -1)
cv2.rectangle(src, (100, 150), (400, 350), (0, 0, 0), -1)
cv2.rectangle(src, (200, 200), (300, 300), (255, 255, 255), -1)
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
# src에 512 x 512 크기의 3-채널 컬러 영상을 생성하고, cv2.rectangle()로 채워진 사각형을 그린다.
# cv2.cvtColor()로 그레이스케일 영상 gray으로 변환한다.

#2
mode = cv2.RETR_LIST
method = cv2.CHAIN_APPROX_SIMPLE;
contours, hierarchy = cv2.findContours(gray, mode, method)
# cv2.drawContours(src, contours, -1, (255,0,0), 3)  
# cv2.findContours()로 윤곽선 contours를 검출한다. mode = cv2.RETR_LIST로 리스트 contours에 len(contours) = 3개의 모든 윤곽선을 검출한다.
# method = cv2.contours[0]은 contours[0].shape = (4, 1, 2)로 4개의 검출된 좌표가 (1, 2) 배열에 저장된다.
# cv2.drawContours(src, contours, -1, (255, 0, 0), 3)는 리스트 contours의 모든 윤곽선을 그린다.

print('len(contours)=', len(contours))
print('contours[0].shape=', contours[0].shape)
print('contours=', contours)

#3
for cnt in contours:
    cv2.drawContours(src, [cnt], 0, (255,0,0), 3)
    
    for pt in cnt: 
        cv2.circle(src, (pt[0][0], pt[0][1]), 5, (0,0,255), -1)
# for 문으로 리스트 contours의 각 윤곽선 cnt를 cv2.drawContours(src, [cnt], 0, (255, 0, 0), 3)로 그리고, for문으로 cnt의 각 좌표pt가 중심인 반지름 5의 원을
# (0, 0, 255)컬러로 그린다.

cv2.imshow('src',  src)
cv2.waitKey()
cv2.destroyAllWindows()


len(contours)= 3
contours[0].shape= (4, 1, 2)
contours= (array([[[200, 200]],

       [[200, 300]],

       [[300, 300]],

       [[300, 200]]], dtype=int32), array([[[ 99, 150]],

       [[100, 149]],

       [[400, 149]],

       [[401, 150]],

       [[401, 350]],

       [[400, 351]],

       [[100, 351]],

       [[ 99, 350]]], dtype=int32), array([[[ 50, 100]],

       [[ 50, 400]],

       [[450, 400]],

       [[450, 100]]], dtype=int32))


### 영역 채우기 · 인페인트 · 거리 계산 · 워터쉐드

1. cv2.floodFill()은 물체의 내부를 특정 값으로 채우고, cv2.impaint()는 영상에서 부분 영역을 삭제하고 주변의 화소 값을 이용하여 채우며, cv2.distanceTransform()은 영상 영역의 내부의 0이 아닌 화소에서, 가장 가까운 0인 화소까지의 거리를 계산한다.
2. cv2.watershed()는 마커 기반 영상 분할을 수행한다.

- cv2.floodFill(image, mask, seedPoint, newVal[, loDiff[, upDiff[, flags ]]])

##### 입력 영상 image에서 seedPoint 점에서 시작하여 물체 내부를 채운다. image가 변경되므로 필요한 경우 원본을 복사해 사용한다.
##### falgs에 지정된 이웃(4 또는 8) 화소 (x', y')를 반복적으로 조사해가며 image(x', y') -loDiff <= image(x, y) + upDiff에 있는 (x, y)점을 새로운 값 newVal로 채워 넣는다.
##### masks는 8비트 단일 채널이며, 크기는 입력 영상 image보다 가로와 세로 2 만큼씩 크다.\
##### flags에 cv2.FLOODFILL_FIXED_RANGE이 추가되면 현재 화소와 seedPoint 사이의 차이를 고려하여 영역을 채우고, cv2.FLOODFILL_RANGE가 설정되지 않으면 이웃 화소들 사이의 차이를 고려하여 영역을 채운다. cv2.FLOODFILL_MASK_ONLY가 설정되면 image를 채우지 않고 mask를 채운다.
##### rect는 채워진 영역의 바운딩 사각형을 반환한다.

- cv2.distanceTransform(src, distanceType, maskSize[, dst ]) -> dst

##### src에서 0이 아닌 화소에서 가장 가까운 화소까지의 거리를 계산하여 실수형 배열 dst에 반환한다.
##### src는 1-채널 8비트의 이진 영상이며, dst는 1-채널 32비트 실수형 배열이다.
##### distanceType은 cv2.DIST_L1, cv2.DIST_L2, cv2.DIST_C의 거리 계산 종류이다.
##### maskSize는 3, 5, cv2.DIST_MASK_PRECISE 마스크 크기가 가능하다.

- cv2.watershed(image, markers) -> markers

##### 마커 기반으로 영상을 분할한다.
##### 8비트 3-채널 컬러 영상 image에 사용자가 대략적으로 32비트 정수 1-채널 markers에 부분 영역을 설정하면 영상을 분할하여 markers배열을 반환한다.
##### 초기에 markers에 주어진 영역의 값을 씨앗(seed)으로 하여 나머지 영역을 분할한다.
##### 반환 배열 markers에 1 이상의 값을 가지며 markers의 값이 같으면 동일 특성을 갖는 분할 영역이며, 영역의 경계 부분은 -1을 갖는다.

#### 실습

In [37]:
## cv2.floodFill() 영역 채우기
# 0708.py
import cv2
import numpy as np

#1
src = np.full((512,512,3), (255, 255, 255), dtype= np.uint8)
cv2.rectangle(src, (50, 50), (200, 200), (0, 0, 255), 2)
cv2.circle(src, (300, 300), 100, (0,0,255), 2)
# 512 x 512 크기의 배경이 (255, 255, 255)인 3-채널 컬러 영상 src를 생성하고 사각형과 원을 그린다.

#2
dst = src.copy()
cv2.floodFill(dst, mask=None, seedPoint=(100,100), newVal=(255,0,0))
# src를 dst에 복사하고, cv2.floodFill()로 dst에 seedPoint = (100, 100)을 시작점으로 사각형 내부를 newVal = (255, 0, 0)색상으로 dst에 채운다.

#3
retval, dst2, mask, rect=cv2.floodFill(dst, mask=None,
                          seedPoint=(300,300), newVal=(0,255,0))
print('rect=', rect)
x, y, width, height = rect
cv2.rectangle(dst2, (x,y), (x+width, y+height), (255, 0, 0), 2)
# cv2.floodFill()로 dst의 seedPint = (300, 300)를 시작점으로 원의 내부를 newVal =  (0, 255, 0) 색상으로 dst에 채운다.
# 원의 내부를 채운 영역의 바운딩 사각형 rect를 이용하여 dst2에 사각형을 그린다.

cv2.imshow('src',  src)
cv2.imshow('dst',  dst)
cv2.waitKey()
cv2.destroyAllWindows()


rect= (202, 202, 197, 197)


In [19]:
## cv2.distanceTransform() 거리 계산
# 0709.py
import cv2
import numpy as np

#1
src = np.zeros(shape=(512,512), dtype=np.uint8)
cv2.rectangle(src, (50, 200), (450, 300), (255, 255, 255), -1)
# src에 512 x 512 크기의 그레이스케일 영상을 생성하고, cv2.rectangle()로 채워진 사각형을 그린다.

#2
dist  = cv2.distanceTransform(src, distanceType=cv2.DIST_L1, maskSize=3)
minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(dist)
print('src:', minVal, maxVal, minLoc, maxLoc)
dst = cv2.normalize(dist, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
ret, dst2 = cv2.threshold(dist, maxVal-1, 255, cv2.THRESH_BINARY)
# src에 cv2.distanceTransform()로 distanceType = cv2.DIST_L1, maskSize = 3를 적용하여 dist에 거리를 계산한다.
# cv2.minMaxLoc(dist)로 계산한 최대값은 maxVal = 51.0이다.
# cv2.normalize()로 dist를 [0, 255]범위로 정규화한다.
# cv2.threshold()로 dist를 thresh = maxVal - 1로 dst2에 임계값을 적용한다.

#3 
gx = cv2.Sobel(dist, cv2.CV_32F, 1, 0, ksize = 3)
gy = cv2.Sobel(dist, cv2.CV_32F, 0, 1, ksize = 3)
mag   = cv2.magnitude(gx, gy)
minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(mag)
print('src:', minVal, maxVal, minLoc, maxLoc)
ret, dst3 = cv2.threshold(mag, maxVal-2, 255, cv2.THRESH_BINARY_INV)
# cv2.Sobel()로 거리 dist에서 그래디언트를 계산하고, 크기 mag를 계산하여 thresh = maxVal - 2, cv2.THRESH_BINARY_INV으로 임계값 영상을 생성

cv2.imshow('src',  src)
cv2.imshow('dst',  dst)
cv2.imshow('dst2',  dst2)
cv2.imshow('dst3',  dst3)
cv2.waitKey()
cv2.destroyAllWindows()


src: 0.0 51.0 (0, 0) (100, 250)
src: 0.0 8.0 (0, 0) (52, 200)


In [27]:
## cv2.watershed() 영상 분할
# 0710.py
import cv2
import numpy as np

#1
# src = cv2.imread('./data/hand.jpg')
src = cv2.imread('./data/flower.jpg')
mask   = np.zeros(shape=src.shape[:2], dtype=np.uint8)
markers= np.zeros(shape=src.shape[:2], dtype=np.int32)
dst = src.copy()
cv2.imshow('dst',dst)
# src는 입력 영상이고 마우스로 지정할 마스크 영역을 지정하고, 윤곽선을 검출한 mask 영상, 윤곽선을 이용하여 워터쉐드 분할을 위한 마커 영상 markers를 생성한다.


#2
def onMouse(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE:
        if flags & cv2.EVENT_FLAG_LBUTTON:
            cv2.circle(param[0], (x, y), 10, (255, 255, 255), -1)
            cv2.circle(param[1], (x, y), 10, (255, 255, 255), -1) 
    cv2.imshow('dst', param[1])    
##cv2.setMouseCallback('dst', onMouse, [mask, dst])
# 마우스 이벤트 핸들러 함수 onMouse()를 정의한다. param[0]은 mask, param[1]은 dst가 전달된다.
# 마우스 왼쪽버튼을 누르고 움직이면 param[0], param[1]에 반지름이 20인 채운 원을 그린다.

#3
mode = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE

while True:
    cv2.setMouseCallback('dst', onMouse, [mask, dst]) #3-1
    
    key = cv2.waitKey(30) # cv2.waitKeyEx(30)
    
    if key == 0x1B: 
        break;
        # ESC키를 누르면 반복문 탈출
        
    elif key == ord('r'): #3-2
        mask[:,:] = 0        
        dst = src.copy()
        cv2.imshow('dst',dst)        
        # 'r'키를 누르면 리셋하기 위하며 mask의 모든 화소를 0으로 초기화하고, src를 dst에 복사하고 'dst' 윈도우에 표시한다.
        
    elif key == ord(' '): #3-3
        contours, hierarchy = cv2.findContours(mask, mode, method)
        print('len(contours)=', len(contours))
        markers[:,:] = 0  
        for i, cnt in enumerate(contours):
            cv2.drawContours(markers, [cnt], 0, i+1, -1)
        cv2.watershed(src,  markers)
        # 'space bar'키를 누르면, mask에 윤곽선을 검출하고, markers를 0으로 초기화하고, cv2.drawContours()로 markers에 윤곽선 contours[i]를 i + 1값으로
        # 채워 넣어, cv2.watershed()의 입력으로 사용한다. cv2.watershed()로 src에서 markers에 표시된 마커 정보를 이용하여 영역을 markers에 분할한다.
        
        #3-4        
        dst = src.copy()
        dst[markers == -1] = [0,0,255] # 경계선
        for i in range(len(contours)): # 분할영역 
            r = np.random.randint(256)
            g = np.random.randint(256)
            b = np.random.randint(256)
            dst[markers == i+1] = [b, g, r]
            # src를 dst에 복사하고, dst[markers == -1] = [0, 0, 255]에 의해 markers에 -1인 경계선을 빨간색 [0, 0, 255]로 변경한다.
            # for 문에서 r, g, b에 [0, 255] 사이의 난수를 생성하여 dst[markers == i+1] = [b, g, r]로 markers == i + 1인 dst의 화소를 [b, g, r] 컬러로 변경한다.

        dst = cv2.addWeighted(src, 0.4, dst, 0.6, 0) # src * 0.4 와 dst * 0.6으로 섞어 dst에 저장하고, 'dst'윈도우에 표시한다.
        cv2.imshow('dst',dst)
        
        
cv2.destroyAllWindows()


len(contours)= 9
len(contours)= 1
len(contours)= 0


In [61]:
## cv2.distanceTransform(), cv2.watershed() 영상 분할
# 0711.py
import cv2
import numpy as np

#1
src = cv2.imread('./data/circles2.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
ret, bImage = cv2.threshold(gray, 0, 255,
                                cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
dist  = cv2.distanceTransform(bImage, cv2.DIST_L1, 3)
dist8 = cv2.normalize(dist, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

cv2.imshow('bImage',bImage)
cv2.imshow('dist8',dist8)
# 입력 영상 src를 그레이스케일 영상 gray로 변환하고, threshold()로 임계값을 이용하여 이진 영상 bImage를 생성한다.
# cv2.distanceTransform()로 bImage에서 거리배열 dist를 계산한다.
# 거리를 보여주기 위해 8비트 영상으로 dist8에 정규화한다.

#2
minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(dist)
print('dist:', minVal, maxVal, minLoc, maxLoc)
mask = (dist > maxVal*0.5).astype(np.uint8)*255
cv2.imshow('mask',mask)
# mask = (dist > maxVal * 0.5)로 거리 dist에서 최대값 maxVal를 이용하여 8비트 mask 영상을 계산한다. dist대신 dist8을 이용할 수 있다.
# 여기서 중요한 점은 이진 영상 bImage에서 겹쳐진 원이 거리 계산을 이용한 mask에서는 분리된 것을 알 수 있다.
# 이렇게 분리하기 위하여 cv2.distanceTransform()을 이용한 것이다.

#3
mode = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE
contours, hierarchy = cv2.findContours(mask, mode, method)
print('len(contours)=', len(contours))

markers= np.zeros(shape=src.shape[:2], dtype=np.int32)
for i, cnt in enumerate(contours):
    cv2.drawContours(markers, [cnt], 0, i+1, -1)
# cv2.findContours()로 mask에서 윤곽선 contours를 검출한다.
# 윤곽선 contours[i]를 markers에 i + 1로 채워 마커를 생성하여, cv2.watershed()로 src에서 markers에 표시된 마커 정보를 이용하여 영역을 markers에 분할한다.

#4
dst = src.copy()
cv2.watershed(src,  markers)

dst[markers == -1] = [0, 0, 255] # 경계선
for i in range(len(contours)): # 분할영역
    r = np.random.randint(256)
    g = np.random.randint(256)
    b = np.random.randint(256)
    dst[markers == i+1] = [b, g, r]
dst = cv2.addWeighted(src, 0.4, dst, 0.6, 0) # 합성        
# src를 dst에 복사하고, dst[markers == -1] = [0, 0, 255]에 의해 markers에 -1인 경계선을 빨간색 [0, 0, 255]로 변경한다.
# for 문에서 r, g, b에 [0, 255] 사이의 난수를 생성하여 dst[markers == i + 1]인 dst의 화소를 [b, g, r]컬러로 변경한다.
# cv2.addWeighted()로 src * 0.4와 dst * 0.6으로 섞어 dst에 저장

cv2.imshow('dst',dst)
cv2.waitKey()
cv2.destroyAllWindows()


dist: 0.0 76.0 (0, 0) (220, 220)
len(contours)= 6


### 피라미드 기반 분할

- cv2.pyrDown(src[, dst[, dstsize[, borderType ]]])

##### 영상 src에 가우시안 필터링하고, dstsize에 주어진 크기로 dst에 축소한다.
##### default는 가로, 세로 각각을 1/2배 크기로 축소한다.

- cv2.pyrUp(src[, dst[, dstsize[, borderType ]]])

##### 영상 src에 가우시안 필터링하고 dstsize에 주어진 크기로 dst에 확대한다.
##### default는 가로, 세로 각각을 2배 크기로 확대한다.

- cv2.pyrMeanShiftFiltering(src, sp, sr[, dst[, maxLevel[, termcrit ]]])

##### 영상 src에서 피라미드 기반 평균이동 필터링을 수행한다.
##### 입력 영상 src는 8비트 3채널 컬러 영상이고, dst는 src와 자료형과 크기가 같은 결과 영상으로, 평균이동 알고리즘에 의해 유사한 컬러 값을 갖는 화소가 같은 값을 갖는다.
##### termcrit는 반복의 종료조건을 최대 반복 횟수 cv2.TERM_CRITERIA_MAX_ITER 또는 cv2.TERM_CRITERIA_COUNT와 오차 cv2.TERM_CRITERIA_EPS로 설정한다.
##### sp >= 1 는 공간 윈도우의 반지름, sr은 컬러 윈도우의 반지름, maxLevel은 피라미드의 최대 레벨이다.
##### src의 화소(X, Y)에 대하여, 공간 윈도우와 컬러 윈도우를 사용하여 반복적으로 meanshift를 수행한다.
##### (x, y)는 공간 윈도우 내의 이웃 좌표이다. RGB, HSV 컬러 모델 등 3-개의 요소를 갖는 컬러 모델이면 모두 가능하다.

#### 실습

In [73]:
# 0712.py
import cv2
import numpy as np
#1
src = cv2.imread('./data/lena.jpg')

down2 = cv2.pyrDown(src)
down4 = cv2.pyrDown(down2)
print('src.shape=', src.shape)
print('down2.shape=', down2.shape)
print('down4.shape=', down4.shape)

#2
up2 = cv2.pyrUp(src)
up4 = cv2.pyrUp(up2)
print('up2.shape=', up2.shape)
print('up4.shape=', up4.shape)

cv2.imshow('down2',down2)
## cv2.imshow('down4',down4)
cv2.imshow('up2',up2)
## cv2.imshow('up4',up4)
cv2.waitKey()
cv2.destroyAllWindows()


src.shape= (512, 512, 3)
down2.shape= (256, 256, 3)
down4.shape= (128, 128, 3)
up2.shape= (1024, 1024, 3)
up4.shape= (2048, 2048, 3)


In [77]:
## cv2.pyrMeanShiftFiltering()영역 검출
# 0713.py
import cv2
import numpy as np
#1
def floodFillPostProcess(src, diff=(2,2,2)):
    img = src.copy()
    rows, cols = img.shape[:2]
    mask   = np.zeros(shape=(rows+2, cols+2), dtype=np.uint8)
    for y in range(rows):
        for x in range(cols):
            if mask[y+1, x+1] == 0:
                r = np.random.randint(256)
                g = np.random.randint(256)
                b = np.random.randint(256)
                cv2.floodFill(img,mask,(x,y),(b,g,r),diff,diff)
    return img
# floodFillPostProcess()는 cv2.floodFill() 함수를 사용하여 src를 복사한 img 영상에서 유사한 영역을 채워 분할한다.
# mask는 img보다 가로, 세로로 2만큼 큰 영상이고, 0인 화소, (x, y)를 찾아, cv2.floodFill()로 채우면, (x, y)의 화소값과 위아래로 diff 차이가 나지 않으면 img의
# 해당 화소는 newVal로 채우고, mask는 1로 채운다.

#2
src = cv2.imread('./data/flower.jpg')
hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV)
dst  = floodFillPostProcess(src)
dst2 = floodFillPostProcess(hsv)
cv2.imshow('src',src)
cv2.imshow('hsv',hsv)
cv2.imshow('dst',dst)
cv2.imshow('dst2',dst2)
# BGR입력 영상 src를 HSV영상 hsv로 변환한다.
# floodFillPostProcess()를 src, hsv에 적용하여 각각 dst, dst2로 영역 분할한다.
# 필터링하지 않은 src, hsv 영상에서 디폴트 차이 diff = (2, 2, 2)에 의한 채우기로 영역 분할한 결과는 매우 많은 영역이 검출된다.

#3
res = cv2.pyrMeanShiftFiltering(src, sp=5, sr=20, maxLevel=4)
dst3 = floodFillPostProcess(res)
# cv2.pyrMeanShiftFiltering()로 src를 sp = 5, sr = 20, maxLevel = 4로 피라미드 평균이동 필터링하여 res에 저장한다.
# floodFillPostProcess()를 res에 적용하여 dst으로 영역 분할한다.

#4
term_crit=(cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 10, 2)
res2=cv2.pyrMeanShiftFiltering(hsv,sp=5,sr=20,maxLevel=4, termcrit=term_crit)
dst4 = floodFillPostProcess(res2)
# cv2.pyMeanShiftFiltering()로 hsv를 sp = 5, sr = 20, maxLevel = 4, 최대반복횟수 = 10, 오차 2의 종료 조건을 적용하여 피라미드 평균 이동 필터링하여 res2에 저장
# floodFillPostProcess()를 res2에 적용하여 dst4로 영역 분할한다. 피라미드 필터를 적용한 결과의 영역 분할 결과는 좀 더 큰 영역으로 분할한다.

cv2.imshow('res',res)
cv2.imshow('res2',res2)
cv2.imshow('dst3',dst3)
cv2.imshow('dst4',dst4)
cv2.waitKey()
cv2.destroyAllWindows()


### K-Means 클러스터링 분할

* K-Means 클러스터링 알고리즘
    * 단계 1
    - 클러스터 개수 k를 고정하고, t = 0으로 초기화한다. K개의 클러스터 C<sup>0</sup><sub>i</sub>, i = 1, ..., K의 평균 m<sup>0</sup><sub>i</sub>, i = 1, ...,K을 임의로 선택한다.
    * 단계 2
    - 클러스터링하려는 데이터 x<sub>j</sub>, j = 1, ..., M 각각에 K개의 클러스터 평균과의 최소거리가 되는 클러스터 C<sup>t</sup><sub>p</sub>로 x<sub>j</sub>을 분류한다.
    * 단계 3
    - 각 클러스터 C<sup>t</sup><sub>i</sub>, i = 1, ..., K에 속한 데이터를 이용하여 새로운 클러스터 평균 m<sub>i</sub><sup>t+1</sup>, i = 1, ..., K를 계산한다.
    * 단계 4
    - t = t + 1로 증가시키고, 만약 t > MAX_ITER 또는 err = Σ|m<sub>i</sub><sup>t+1</sup> - m<sub>i</sub><sup>t+1</sup>| < EPS이면 중지하고, 그렇지 않으면 단계 2, 단계 3을 반복한다.

- cv2.kmeans(data, K, bestLabels, criteria, attempts, flags[, centers ]) -> retval, bestlabels, centers

##### data는 클러스터링을 위한 데이터이다. 각 샘플 데이터는 data의 행에 저장된다. K는 클러스터의 개수이고, bestLabels는 각 샘플의 클러스터 번호를 labels에 저장한다.
##### criteria는 종료조건으로 최대 반복회수와 각 클러스터의 중심이 오차 이내로 움직이면 종료한다. cv2.TERM_CRITERIA_MAX_ITER 또는 cv2.TERM_CRITERIA_COUNT와 오차 cv2.TERM_CRITERIA_EPS로 설정한다.
##### attempts는 알고리즘을 시도하는 횟수로, 서로 다른 시도 횟수 중 최적의 레이블링 결과를 bestLabels에 저장하여 반환한다. centers는 클러스터의 중심을 각 행에 저장하여 반환한다.
##### flags는 K 개의 클러스터 중심을 초기화하는 방법을 명시한다. cv2.KMEANS_RANDOM_CENTERS이면 난수를 사용하여 임의로 설정한다. cv2.KMEANS_PP_CENTERS이면 Arthur and Vassilvitskii에 의해 제안 방법을 사용한다. cv2.KMEANS_USE_INITIAL_LABELS이면, 처음 시도에서는 사용자가 제공한 레이블을 사용하고, 다음 시도부터는 난수를 이용하여 임의로 설정한다.
##### 클러스터링 밀집도를 계산하여 retval에 반환한다.

#### 실습

In [12]:
## cv2.kmeans() 컬러 클러스터링 영역 검출
# 0714.py
import cv2
import numpy as np
#1
# src = cv2.imread('./data/hand.jpg')
src = cv2.imread('./data/flower.jpg')
hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV)

# data = src.reshape((-1,3)).astype(np.float32) # BGR 클러스터링
data = hsv.reshape((-1,3)).astype(np.float32) # HSV 클러스터링
# 컬러 입력 영상 src를 HSV 컬러 영상 hsv로 변환한다. 입력 영상 src 또는 hsv의 각 화소의 컬러가 data의 행에 배치되도록 모양을 변환한다.
# data.shape = (230400, 3)이다.

#2
K = 2
term_crit=(cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
ret, labels, centers = cv2.kmeans(data, K, None, term_crit, 5,
                                  cv2.KMEANS_RANDOM_CENTERS)
print('centers.shape=', centers.shape)
print('labels.shape=', labels.shape)
print('ret=', ret)
# cv2.kmeans()로 'hand.jpg'는 K = 2, 'SegmentTest.jpg'는 K = 5로 클러스터링 한다.
# centers.shape = (2, 3)로 centers에 K개의 클러스터 중심점을 반환한다. labels.shape = (230400, 1)로 labels는 각 데이터 점의 클러스터 번호를 반환한다.
# ret는 클러스터 응집도를 반환한다.

#3
centers = np.uint8(centers)
res   = centers[labels.flatten()]
dst  = res.reshape(src.shape)

labels2 = np.uint8(labels.reshape(src.shape[:2]))
print('labels2.max()=', labels2.max())
dst   = np.zeros(src.shape, dtype=src.dtype)
for i in range(K): # 분할영역 표시
    r = np.random.randint(256)
    g = np.random.randint(256)
    b = np.random.randint(256)
    dst[labels2 == i] = [b, g, r]

# centers를 np.uint8 자료형으로 변환하고, res = centers[labels.flatten()]로 res에 레이블에 대한 클러스터 중심으로 변환한 res를 생성한다. res.shape = (230400, 3)이다.
# res를 src와 같은 영상 모양으로 변환한다. 주석 처리 부분은 label2의 각 클러스터 번호에 난수로 생성한 컬러를 지정하여 dst 영상을 생성한다.
    
cv2.imshow('dst',dst)
cv2.waitKey()
cv2.destroyAllWindows()


centers.shape= (2, 3)
labels.shape= (261120, 1)
ret= 1274532890.7591445
labels2.max()= 1


### 연결 요소 검출

- cv2.connectedComponents(image[, labels[, connectivity[, ltype]]]) -> retval, labels

##### image, 8비트 1-채널 입력 영상이다. labels는 연결 요소 정보를 갖는 입력 영상과 같은 크기의 출력 레이블이다.
##### connectivity는 화소의 이웃 연결성으로 4 또는 8이다. Itype은 출력 labels의 자료형으로 cv2.CV_32S 또는 cv2.CV_16U이다.

- cv2.connectedComponentsWithStats(image[, labels[, stats[, centroids[, connectivity[, ltype]]]]])

##### image는 8비트 1-채널 입력 영상이다. labels는 연결 요소 정보를 갖는 입력 영상과 같은 크기의 출력 레이블이다.
##### stats는 각 레이블에 대해 5열에 바운딩 사각형의 (left, top, width, height, area) 통계정보를 갖는다. 0행은 배경 레이블 정보이다.
##### centroids는 각 레이블의 중심좌표이다.
##### connectivity는 화소의 이웃 연결성으로 4 또는 8이다. ltype은 출력 labels의 자료형으로 cv2.CV_32S 또는 cv2.CV_16U이다.

#### 실습

In [20]:
## 레이블링 1(임계값 이진 영상): cv2.connectedComponents()
# 0715.py
import cv2
import numpy as np

#1
src = cv2.imread('./data/circles.jpg')
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)
ret, res = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV)
# 컬러 입력 영상 src를 그레이스케일 영상 gray로 변환한다. 입력 영상에서 원이 검은색, 배경이 흰색이어서 cv2.THRESH_BINARY_INV로 입계값 128을 적용하여 이진영상 생성

#2
ret, labels = cv2.connectedComponents(res)
print('ret=', ret)
# cv2.connectedComponents()로 이진영상 res를 레이블링하여 레이블 개수는 배경을 포함하여 ret = 4이고, 레이블정보 labels를 생성한다.
# 검출된 원의 개수 ret - 1이다.

#3
dst   = np.zeros(src.shape, dtype=src.dtype)
for i in range(1, ret): # 분할영역 표시
    r = np.random.randint(256)
    g = np.random.randint(256)
    b = np.random.randint(256)
    dst[labels == i] = [b, g, r]
# labels에서 배경 레이블(0)은 제외하고, 1에서부터 ret - 1까지의 레이블 영역을 난수로 생성한 같은 컬러로 지정하여 dst영상을 생성한다.

cv2.imshow('res',  res)
cv2.imshow('dst',  dst) 
cv2.waitKey()
cv2.destroyAllWindows()


ret= 4


In [19]:
# 0716.py
import cv2
import numpy as np

#1
src = cv2.imread('./data/circles.jpg')
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)
ret, res = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV)
# 컬러 입력 영상 src를 그레이스케일 영상 gray로 변환한다. cv2.THRESH_BINARY_INV로 임계값 128을 적용하여 이진영상 res를 생성한다.

#2
ret, labels, stats, centroids = cv2.connectedComponentsWithStats(res)
print('ret =', ret)
print('stats =', stats)
print('centroids =', centroids)
# cv2.connectedComponentsWithStats()로 이진 영상 res를 레이블링하여 레이블 개수 ret, 레이블 정보 labels, 통계정보 stats, 중심점 centroids를 계산한다.

#3
dst   = np.zeros(src.shape, dtype=src.dtype)
for i in range(1, int(ret)): # 분할영역 표시
    r = np.random.randint(256)
    g = np.random.randint(256)
    b = np.random.randint(256)
    dst[labels == i] = [b, g, r]
# labels에서 배경 레이블(0)은 제외하고, 1에서부터 ret -1 까지의 레이블 영역을 난수로 생성한 같은 컬러로 채운다.

#4    
for i in range(1, int(ret)):
    x, y, width, height, area = stats[i]
    cv2.rectangle(dst, (x,y), (x+width, y+height), (0, 0, 255), 2)

    cx, cy = centroids[i]
    cv2.circle(dst, (int(cx), int(cy)), 5, (255,0,0), -1)
# 레이블 i의 통계정보 stats[i]를 이용하여 바운딩 빨간색으로 사각형을 그리고, 중심점 centroids[i]를 이용하여 파란색으로 원을 그린다.

cv2.imshow('src',  src)
cv2.imshow('dst',  dst) 
cv2.waitKey()
cv2.destroyAllWindows()


ret = 4
stats = [[     0      0    512    512 222719]
 [   308     86    125    125  12281]
 [   153    145    152    152  18152]
 [   292    338    107    107   8992]]
centroids = [[247.77339607 258.80937863]
 [370.         148.        ]
 [228.5        220.50534376]
 [345.00077847 390.99477313]]
